In [ ]:
import os
import sys
import requests

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
from bs4 import BeautifulSoup

from packages.utils import get_total_stats



current_dir = os.path.dirname(os.path.abspath('/Users/maukanmir/Documents/Machine-Learning/Web-Scraping-Code/Ball-Dont-Lie-API/advanced_stats.ipynb'))
project_root = os.path.join(current_dir, '..')
sys.path.insert(0, project_root)

In [ ]:
dotenv_path = ("/Users/maukanmir/Documents/Machine-Learning/Web-Scraping-Code/Ball-Dont-Lie-API/dot.env")
load_dotenv(dotenv_path)

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
TABLE_NAME = "Total_Stats"

DB_USER = os.getenv("DB_USER")

engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}')

In [ ]:
years = [str(year) for year in range(1980, 2025)]
all_data= pd.DataFrame()

for year in years:
  base_url = f"https://www.basketball-reference.com/leagues/NBA_{year}.html"
  response = requests.get(base_url)
  soup = BeautifulSoup(response.text, "html.parser")
  try:
    year = int(year)
    df = get_total_stats(soup, year)
    all_data = pd.concat([all_data, df])
  except Exception as error:
    print(f" The error is {error}")

In [ ]:
try:
    all_data.to_sql(TABLE_NAME, engine, if_exists='replace', index=False)
    print("Data successfully written to the database.")
except Exception as e:
    print(f"Database operation failed. Error: {e}")

In [ ]:
def per_game_stats(soup, year):
    table = soup.find("table", {"id": "per_game-team"})

    # Extract the headers
    headers = [th.text.strip() for th in table.find("thead").find_all("th")]

    # Extract the rows from the tbody
    rows = table.find("tbody").find_all("tr")

    # Extract the data for each row
    data = []
    for row in rows:
        cells = row.find_all("td")
        row_data = [cell.text.strip() for cell in cells]
        data.append(row_data)

    # Create a DataFrame from the parsed data
    df = pd.DataFrame(data, columns=headers[1:])
    df
    df["Season"] = year
    return df
